In [10]:
import numpy as np
import pandas as pd
import string
from time import time
from matplotlib import pyplot as plt

directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1),  (1, 0), (1, 1)]

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_word = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_word

    def starts_with(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

trie = Trie()
with open('dictionary_actual_game.txt', 'r') as file:
    for word in file.read().splitlines():
        trie.insert(word)
        
keys = list(string.ascii_lowercase)
score = [1,3,3,2,1,4,2,4,3,8,5,5,3,2,1,3,10,1,1,1,2,4,4,8,4,10]
letter_to_int = dict(zip(keys, range(1, 27)))
letter_to_int["N"] = 0

dictionary_df = pd.read_csv("dictionary_df2.csv")
dictionary_df.loc[160413, "word"] = "null"
dictionary_df.loc[154353, "word"] = "nan"
dictionary_df = dictionary_df[dictionary_df.columns[1:]]
score_dict = dict(zip(dictionary_df["word"], dictionary_df["score"]))

def tiles_remove(tiles, board):
    board[tiles[:, 0], tiles[:, 1]] = "N"
    return board

def gravity(board_1):
    # Gravity down
    row_index, col_index = np.where(board_1 == "N")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        column = board_1[:r + 1, c].copy()
        if r > 0:
            column[1:] = column[:-1]
            column[0] = "N"
        board_1[:r + 1, c] = column
    # Gravity right
    row_index, col_index = np.where(board_1 == "N")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        row = board_1[r, :c + 1].copy()
        if c > 0:
            row[1:] = row[:-1]
            row[0] = "N"
        board_1[r, :c + 1] = row
    return board_1

class WordSoup_state:
    def __init__(self, board, S=0, word_lst=[], depth=0):
        self.board = board
        self.score = S
        self.word_lst = word_lst
        self.depth = depth
        self.board_array = np.array(board)

    def get_legal_actions(self):
        rows, cols = len(self.board), len(self.board[0])
        result = []

        def dfs(i, j, path, indices, visited, node):
            if node.is_word:
                result.append((path, indices))  # Store the word and its indices
            for dx, dy in directions:
                x, y = i + dx, j + dy
                if 0 <= x < rows and 0 <= y < cols and (x, y) not in visited:
                    char = self.board[x][y]
                    if char in node.children:
                        visited.add((x, y))
                        dfs(x, y, path + char, indices + [(x, y)], visited, node.children[char])
                        visited.remove((x, y))

        for i in range(rows):
            for j in range(cols):
                char = self.board[i][j]
                if char in trie.root.children:
                    dfs(i, j, char, [(i, j)], {(i, j)}, trie.root.children[char])

        return result

    def is_game_over(self):
        found_words = self.get_legal_actions()
        if len(found_words) > 0:
            return False
        return True

    def game_result(self, Score):
        if self.score >= Score:
            return 1
        return -1

    def move(self, action):
        word, indices = action
        _ = tiles_remove(np.array(indices), self.board_array.copy())
        new_board = gravity(_)
        new_score = self.score + score_dict[word]
        if np.sum(new_board != "N") == 0:
            new_score += 500
        new_board = new_board.tolist()
        new_word_lst = self.word_lst.copy()
        new_word_lst.append(word)

        return WordSoup_state(new_board, new_score, new_word_lst, self.depth+1)

In [11]:
def ExhaustiveSearch(state, step=1):
    L = 0
    for l in state.board:
        for i in l:
            L+=1
    untried_actions = state.get_legal_actions()
    untried_actions_sorted = sorted(untried_actions, key=lambda x: score_dict[x[0]], reverse=True)
    best_score_so_far = score_dict[untried_actions_sorted[0][0]]
    best_ = []
    for action in untried_actions_sorted:
        if score_dict[action[0]]==best_score_so_far:
            best_.append(action)
        else:
            break
    result = [state.move(action) for action in best_]
    if step == 1:
        return result
    else:
        queue = [state.move(action) for action in untried_actions_sorted]
        best = []
    
    while queue:
        state = queue.pop(0)
        y = best_score_so_far - state.score
        untried_actions = state.get_legal_actions()
        if len(untried_actions) == 0:
            continue
        length = 0
        for l in state.board:
            for i in l:
                if i != "N":
                    break
                length+=1
        
        def Score_Cal(x):
            if len(x[0])==(L-length):
                return score_dict[x[0]]+500
            else:
                return score_dict[x[0]]
        untried_actions_sorted = sorted(untried_actions, key=lambda x: Score_Cal(x), reverse=True)
        s_ = Score_Cal(untried_actions_sorted[0])
        if s_ > y:
            best_score_so_far = state.score + s_
            best = []
            for x in untried_actions_sorted:
                if Score_Cal(x)==s_:
                    best.append((state, x))
                else:
                    break
        if state.depth < step - 1:
            _ = [state.move(action) for action in untried_actions_sorted]
            if _:
                queue = _ + queue
            else:
                continue
    if best:
        result = [state.move(action) for state, action in best]
    return result

In [111]:
%%time
t0 = time()
depth = selected_state.depth + 2
queue = ExhaustiveSearch(selected_state, step=depth)
t1 = time()
print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in queue]},\nScore: {[s.score for s in queue]},\nTime: {t1-t0} s")
R3 = []

Horizon: 7,
Words chosen: [['quinches', 'queint', 'prolix', 'ixtle', 'qat', 'supermale', 'larkishness'], ['quinches', 'queint', 'prolix', 'ixtle', 'qat', 'supermale', 'larkishness']],
Score: [1011, 1011],
Time: 84.20859694480896 s
CPU times: user 1min 23s, sys: 295 ms, total: 1min 23s
Wall time: 1min 24s


In [87]:
while queue:
    t0 = time()
    state = queue.pop(0)
    if state.is_game_over():
        R3.append((state.word_lst, state.score))
        continue
    else:
        depth = state.depth
        if depth <= 8:
            depth += 2
            try:
                selected_lst = ExhaustiveSearch(state, step=depth)
                queue = selected_lst + queue
                t1 = time()
                print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
            except:
                selected_lst = ExhaustiveSearch(state, step=1)
                t1 = time()
                print(f"Horizon: {depth-1},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
        else:
            if np.sum(state.board_array!="N") <= 15:
                depth += 5
                try:
                    selected_lst = ExhaustiveSearch(state, step=depth)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                except:
                    selected_lst = ExhaustiveSearch(state, step=depth-1)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth-1},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                else:
                    selected_lst = ExhaustiveSearch(state, step=1)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth-2},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                continue
            depth += 3
            try:
                selected_lst = ExhaustiveSearch(state, step=depth)
                queue = selected_lst + queue
                t1 = time()
                print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
            except:
                selected_lst = ExhaustiveSearch(state, step=depth-1)
                queue = selected_lst + queue
                t1 = time()
                print(f"Horizon: {depth-1},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
            else:
                selected_lst = ExhaustiveSearch(state, step=1)
                queue = selected_lst + queue
                t1 = time()
                print(f"Horizon: {depth-2},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")


result = sorted(R3, key=lambda x: x[1], reverse=True)[0]

Horizon: 8,
Words chosen: [['quinches', 'queint', 'prolix', 'ixtle', 'qat', 'supermale', 'larkishness', 'annotating', 'forebody']],
Score: [1307],
Time: 11.93931794166565 s
CPU times: user 11.9 s, sys: 40.4 ms, total: 11.9 s
Wall time: 11.9 s


In [12]:
B_lst = []
filenames_lst = [f"game_0{i}_grid.txt" for i in range(1, 10)]
filenames_lst.append("game_10_grid.txt")
for filename in filenames_lst:
    with open("../sample_human_games/"+filename, "r") as file:
        B_lst.append(file.read().lower().splitlines())
_ = [[list(b) for b in B] for B in B_lst]
state_lst = [WordSoup_state(b) for b in _]

In [13]:
rare_lst = []
for b in _:
    rare_letters = 0
    for row in b:
        rare_letters += row.count("j")
        rare_letters += row.count("q")        
        rare_letters += row.count("x")
    rare_lst.append(rare_letters)

In [14]:
rare_lst

[6, 3, 1, 1, 5, 2, 7, 5, 4, 2]

# HGPM

In [15]:
def HGPM(state):
    rare_letters = 0
    for row in state.board:
        rare_letters += row.count("j")
        rare_letters += row.count("q")        
        rare_letters += row.count("x")
    selected_state = state
    
    if rare_letters > 0:
        for i in range(rare_letters):
            t0 = time()
            actions = selected_state.get_legal_actions()
            if not actions:
                t1 = time()
                print(f"No actions found, time elapse: {t1-t0} s")
                break
            actions_rare = [(word, indice) for word, indice in actions if ("j" in word) or ("q" in word) or ("x" in word)]
            if not actions_rare:
                t1 = time()
                print(f"No actions with rare letters found, time elapse: {t1-t0} s")
                break
            else:
                actions_sorted = sorted(actions_rare, key=lambda x: score_dict[x[0]], reverse=True)
                selected_action = actions_sorted[0]
                selected_state = selected_state.move(selected_action)
                t1 = time()
                print(f"Horizon: {selected_state.depth},\nWords chosen: {selected_state.word_lst},\nScore: {[selected_state.score]},\nTime: {t1-t0} s")
                
                
    t0 = time()
    depth = selected_state.depth + 2
    queue = ExhaustiveSearch(selected_state, step=depth)
    t1 = time()
    print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in queue]},\nScore: {[s.score for s in queue]},\nTime: {t1-t0} s")
    R3 = []

    while queue:
        t0 = time()
        state = queue.pop(0)
        if state.is_game_over():
            R3.append((state.word_lst, state.score))
            continue
        else:
            depth = state.depth
            if depth <= 8:
                depth += 2
                try:
                    selected_lst = ExhaustiveSearch(state, step=depth)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                except:
                    selected_lst = ExhaustiveSearch(state, step=1)
                    t1 = time()
                    print(f"Horizon: {depth-1},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
            else:
                if np.sum(state.board_array!="N") <= 15:
                    depth += 5
                    try:
                        selected_lst = ExhaustiveSearch(state, step=depth)
                        queue = selected_lst + queue
                        t1 = time()
                        print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                    except:
                        selected_lst = ExhaustiveSearch(state, step=depth-1)
                        queue = selected_lst + queue
                        t1 = time()
                        print(f"Horizon: {depth-1},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                    else:
                        selected_lst = ExhaustiveSearch(state, step=1)
                        queue = selected_lst + queue
                        t1 = time()
                        print(f"Horizon: {depth-2},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                    continue
                depth += 3
                try:
                    selected_lst = ExhaustiveSearch(state, step=depth)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                except:
                    selected_lst = ExhaustiveSearch(state, step=depth-1)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth-1},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")
                else:
                    selected_lst = ExhaustiveSearch(state, step=1)
                    queue = selected_lst + queue
                    t1 = time()
                    print(f"Horizon: {depth-2},\nWords chosen: {[s.word_lst for s in selected_lst]},\nScore: {[s.score for s in selected_lst]},\nTime: {t1-t0} s")


    result = sorted(R3, key=lambda x: x[1], reverse=True)[0]
    comb, score = result
    _ = 0
    R = []
    for word in comb:
        _ += len(word)
    if _ == 108:
        R.append((comb, score, 1))
    else:
        R.append((comb, score, 0))
    return R

In [16]:
%%time
result = []
n = 1
for state in state_lst:
    print("-"*50)
    print(f"Sample Board{n}:")
    result.append(HGPM(state))
    n+=1
    print("-"*50)
result

--------------------------------------------------
Sample Board1:
Horizon: 1,
Words chosen: ['mijnheer'],
Score: [184],
Time: 0.04235219955444336 s
Horizon: 2,
Words chosen: ['mijnheer', 'jinnis'],
Score: [298],
Time: 0.02548813819885254 s
Horizon: 3,
Words chosen: ['mijnheer', 'jinnis', 'buqsha'],
Score: [424],
Time: 0.024045944213867188 s
Horizon: 4,
Words chosen: ['mijnheer', 'jinnis', 'buqsha', 'detox'],
Score: [489],
Time: 0.024274110794067383 s
Horizon: 5,
Words chosen: ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs'],
Score: [584],
Time: 0.019840002059936523 s
No actions with rare letters found, time elapse: 0.02049088478088379 s
Horizon: 7,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable']],
Score: [1025, 1025],
Time: 61.69648885726929 s
Horizon: 9,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'mon

Horizon: 13,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'gristle', 'flawy', 'outgazed'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'gristle', 'flawy', 'outgazed']],
Score: [1811, 1811],
Time: 175.9019751548767 s
Horizon: 11,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled']],
Score: [1591, 1591],
Time: 175.90903115272522 s
Horizon: 14,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled', 'feta', 'guy', 'eulogize']],
Score: [1843],
Time: 6.004374027252197 s
Horizon: 12,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'det

Horizon: 14,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled', 'feta', 'guy', 'eulogize']],
Score: [1843],
Time: 5.872554063796997 s
Horizon: 12,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled', 'giglet'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled', 'outlay']],
Score: [1675, 1675],
Time: 5.874844074249268 s
Horizon: 15,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled', 'giglet', 'fez', 'eau', 'yuke'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'promisee', 'wrastled', 'giglet', 'fez', 'eau', 'yuko']],
Score: [1780, 1780],
Time: 0.1189119815826416 s
Horizon: 13,
Words ch

Horizon: 13,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey', 'fique'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey', 'fique'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey', 'fique']],
Score: [1800, 1800, 1800],
Time: 58.29154920578003 s
Horizon: 11,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew']],
Score: [1568, 1568],
Time: 58.29719805717468 s
Horizon: 14,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', '

Horizon: 14,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey', 'fique', 'outage']],
Score: [1848],
Time: 5.012094974517822 s
Horizon: 12,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey']],
Score: [1700, 1700],
Time: 5.014658212661743 s
Horizon: 15,
Words chosen: [['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey', 'fique', 'outage', 'eke'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'machineable', 'behotes', 'monologuist', 'partlets', 'misgrew', 'tolzey', 'fique', 'outage', 'eek'], ['mijnheer', 'jinnis', 'buqsha', 'detox', 'qophs', 'dilatory', 'm

...

Horizon: 15,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'hizz', 'fripon', 'hoddle', 'dev', 'brrr']],
Score: [1740],
Time: 0.6056489944458008 s
Horizon: 13,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'hizz', 'fripon', 'hoddle'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'hizz', 'fripon', 'hoddle']],
Score: [1695, 1695],
Time: 0.606968879699707 s
Horizon: 16,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'hizz', 'fripon', 'hoddle', 'dev', 'brrr']],
Score: [1740],
Time: 0.012686967849731445 s
Horizon: 14,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'hizz', 'fripon', 'hoddle', 'peh'], ['jam

Horizon: 15,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'brrr', 'pope', 'dirndl']],
Score: [1727],
Time: 0.4836421012878418 s
Horizon: 13,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'devoid']],
Score: [1659],
Time: 0.48488593101501465 s
Horizon: 16,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'devoid', 'brrr'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'devoid', 'brrr'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'devoid', 'brrr'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzo

Horizon: 15,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'pip', 'dehorn', 'brr']],
Score: [1689],
Time: 0.3131680488586426 s
Horizon: 13,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'poind'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'ephod']],
Score: [1636, 1636],
Time: 0.31456708908081055 s
Horizon: 16,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'poind', 'dev', 'brrr'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hohed', 'poind', 'dev', 'brrr']],
Score: [1681, 1681],
Time: 0.014026880264282227 s
Horizon: 14,
Words chosen: 

Horizon: 14,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'doh', 'dripped', 'clover']],
Score: [1737],
Time: 8.860296964645386 s
Horizon: 12,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hoddin'], ['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hoddin']],
Score: [1605, 1605],
Time: 8.862573862075806 s
Horizon: 15,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hoddin', 'drove', 'chelp', 'brrr']],
Score: [1754],
Time: 1.0218260288238525 s
Horizon: 13,
Words chosen: [['jamlike', 'orixas', 'jambart', 'jaconet', 'lanx', 'ratchet', 'illegalised', 'matzoh', 'overhunted', 'piroghi', 'fizz', 'hoddin', 'evohe']],
Score: [1660],
Time: 1.0232198238372803 s


KeyboardInterrupt



In [17]:
result

[[(['mijnheer',
    'jinnis',
    'buqsha',
    'detox',
    'qophs',
    'dilatory',
    'machineable',
    'behotes',
    'monologuist',
    'martlet',
    'sweater',
    'gospelize',
    'gift',
    'queue',
    'kyu',
    'dow',
    'hue'],
   2450,
   1)],
 [(['relax',
    'galletas',
    'pedestrianises',
    'kapoks',
    'homoeopathies',
    'sulph',
    'constituent',
    'yttric',
    'wavesons',
    'waqfs',
    'truck',
    'dawd',
    'koph',
    'thrombi',
    'doo'],
   1788,
   0)],
 [(['waxier',
    'spiritousness',
    'windlassing',
    'rullocks',
    'fluidizer',
    'magilp',
    'workarounds',
    'dividual',
    'chikhors',
    'awhape',
    'whump',
    'crupper',
    'faze',
    'ram',
    'rho'],
   2674,
   1)],
 [(['quannets',
    'chalkface',
    'aftershaves',
    'crotalum',
    'photogenic',
    'picayune',
    'bloodwood',
    'copaivas',
    'grivation',
    'zaffres',
    'yuky',
    'tec',
    'dot',
    'chugged',
    'gur'],
   1967,
   0)]]

In [18]:
scores_lst = [i[0][1] for i in result]
is_solve_lst = [i[0][2] for i in result]

In [19]:
np.mean(scores_lst)

2219.75

In [20]:
np.std(scores_lst)

356.9484381531876

In [21]:
cumulative_score_game = []
for i in range(10):
    _ = [0]
    cumulative_score = 0
    for word in result[i][0][0]:
        cumulative_score += score_dict[word]
        _.append(cumulative_score)
    _ = _[:-1] + [result[i][0][1]]
    cumulative_score_game.append(_)

IndexError: list index out of range

In [ ]:
cumulative_score_game

In [ ]:
score_human = [
    [21,133,91,120,84,45,102,112,133,78,70,108,75,28,28,36,68,18,39,20,30,521],
    [16,15,90,120,144,35,115,70,32,70,15,32,30,9,324,114,36,35,18,32,18,12,528],
    [85,27,16,432,36,324,84,153,75,84,65,36,28,68,45,70,36,21,60,540], 
    [18,28,24,45,560,21,21,171,55,60,119,24,44,44,50,42,27,44,18,18,18,24,21,516],
    [70,21,20,15,108,40,40,30,15,646,77,70,108,52,27,36,48,32,39,24,52,24],
    [24,18,24,21,36,56,432,40,171,78,55,55,108,28,30,60,45,24,32,44,21,15,33],
    [28,68,21,40,21,60,85,70,15,21,594,161,36,27,52,32,85,42,30,36,39,9,12,12,530],
    [12,30,28,40,44,15,55,36,24,32,40,75,45,15,15,20,44,784,21,15,24,18,15,39,18,518],
    [36,96,78,18,50,72,56,66,90,18,210,144,40,56,64,12,12,60,32,24,60,21,527],
    [36,102,44,32,12,135,24,40,102,18,44,21,36,20,40,21,21,18,840,40,60,32,536]
]
cumulative_score_human = []
for score_lst in score_human:
    _ = [0]
    cumulative_score = 0
    for score in score_lst:
        cumulative_score += score
        _.append(cumulative_score)
    cumulative_score_human.append(_)

In [ ]:
for i in range(10):
    plt.figure();
    plt.plot(range(len(cumulative_score_game[i])), cumulative_score_game[i], label="HGPM");
    plt.axhline(y=cumulative_score_game[i][-1], color='r', linestyle='--');
    plt.plot(range(len(cumulative_score_human[i])), cumulative_score_human[i], label="Human experts");
    plt.axhline(y=cumulative_score_human[i][-1], color='g', linestyle='--');
    plt.xlabel("Horizon");
    plt.ylabel("Cumulative Score");
    plt.legend();
    plt.savefig(f"Plots/Figure{i}.pdf")